In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks

from tensorflow.python.data.experimental import AUTOTUNE

from tripletdataset import TripletDataset


2022-05-22 17:04:51.274332: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
dataset = TripletDataset(root_path='/mnt/d/cardimagescans/', set_codes=['khm', 'isd', 'sta', 'ogw'], batch_size=16)

/mnt/d/cardimagescans/ ['khm', 'isd', 'sta', 'ogw']
loading 932 images


2022-05-22 17:04:53.822269: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-22 17:04:53.862479: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-22 17:04:53.862543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce GTX 1660 SUPER computeCapability: 7.5
coreClock: 1.83GHz coreCount: 22 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2022-05-22 17:04:53.862578: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-22 17:04:53.873777: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05-22 17:04:53.884344: I tensorflow/stream_ex

In [3]:
def tripletloss(model_anchor, model_positive, model_negative, margin=0.5):
    distance1 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_positive, 2), 1, keepdims=True))
    distance2 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_negative, 2), 1, keepdims=True))
    return tf.reduce_mean(tf.maximum(distance1 - distance2 + margin, 0))

def tripletloss_wrapper(loss_list, margin = 0.5):
    return (tripletloss(loss_list[0], loss_list[1], loss_list[2], margin))

def make_triplet_loss_network(input_size):
    
    input_anker = ks.Input(input_size)
    input_positive = ks.Input(input_size)
    input_negative = ks.Input(input_size)

    model = ks.Sequential()
    model.add(
        ks.applications.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_size,
        pooling='max'
    )
    )
    model.add(ks.layers.Flatten())
    model.add(ks.layers.Dense(265, activation='sigmoid'))

    encoded_anker = model(input_anker)
    encoded_positive = model(input_positive)
    encoded_negative = model(input_negative)

    triplet_model = ks.Model(inputs=[input_anker, input_positive, input_negative], outputs=[encoded_anker, encoded_positive, encoded_negative])

    #loss = tripletloss(encoded_anker, encoded_positive, encoded_negative)

    

    # similarity_layer = ks.layers.Lambda(lambda x: tripletloss(x[0], x[1], x[2]))
    # similarity_score = similarity_layer([encoded_anker, encoded_positive, encoded_negative])

    return triplet_model

In [4]:
triplet_model = make_triplet_loss_network([350,250,3])

triplet_model.summary()

triplet_model.compile(optimizer=ks.optimizers.Adam(), loss=tripletloss_wrapper)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 350, 250, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 350, 250, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 350, 250, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 265)          14850633    input_1[0][0]                    
                                                                 input_2[0][0]         

In [ ]:
triplet_model.fit()

In [ ]:
model = make_triplet_loss_network([350,250,3])

model.summary()

In [ ]:
anker, positive, negative = dataset.get_triplet()

model.evaluate()

In [ ]:
# load tfrecords dataset
image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, input_shape)
    return image, parsed_example["class"]

def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

def tripletloss(model_anchor, model_positive, model_negative, margin=2):
    distance1 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_positive, 2), 1, keepdims=True))
    distance2 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_negative, 2), 1, keepdims=True))
    return tf.reduce_mean(tf.maximum(distance1 - distance2 + margin, 0))

In [ ]:
dataset = load_and_extract_images(data_path)

In [ ]:
def triplet_input_fn(dataset):
    double_set = tf.data.Dataset(dataset.batch(2))
    return double_set

triplet_set = triplet_input_fn(dataset)

In [ ]:
from matplotlib.pyplot import imshow
test_set = train_dataset.take(1)
for image, label in test_set:
    print(image.shape)
    imshow(image)

In [ ]:
convnet = ks.Sequential()
convnet.add(ks.layers.Input(input_shape))
convnet.add(ks.layers.Conv2D(32, [7,7]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Conv2D(64, [5,5]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Conv2D(128, [3,3]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Flatten())
convnet.add(ks.layers.Dense(256, activation=None))
convnet.add(ks.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))

In [ ]:
convnet.summary()

In [ ]:
convnet.compile(optimizer='Adam', loss=tripletloss, metrics=['accuracy'])

In [ ]:
for epoch in range(10):
    convnet.fit(dataset, batch_size=8)